## Spark Setup
We create a SparkSession to process batch CSvs and micro-batch inventory streams.
The app name and master configuration are from "config.yaml".

In [10]:
from pathlib import Path

project_root = Path.cwd().parent

In [11]:
import yaml
from pyspark.sql import SparkSession


# Load config.yaml
with open("../config.yaml", 'r') as f:
    cfg = yaml.safe_load(f)

 # Initialize Spark Session
spark = (SparkSession.builder
    .appName(cfg["spark"]["app_name"])
    .master(cfg["spark"]["master"])
    .config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/retail_db")
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.5.0")
    .getOrCreate())
spark

In [12]:
print(spark.sparkContext.getConf().get("spark.jars"))

file:///home/prabin/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.12-10.5.0.jar,file:///home/prabin/.ivy2/jars/org.mongodb_mongodb-driver-sync-5.1.4.jar,file:///home/prabin/.ivy2/jars/org.mongodb_bson-5.1.4.jar,file:///home/prabin/.ivy2/jars/org.mongodb_mongodb-driver-core-5.1.4.jar,file:///home/prabin/.ivy2/jars/org.mongodb_bson-record-codec-5.1.4.jar


## Data Ingestion
We are loading batch load of historical sales and customers data and a stimulated mini-stream files drop for inventory sensor events.

In [13]:
# Configuring path for global use
sales_path = project_root/ "data" / "raw" /"sales_data.csv"
customers_path = project_root/ "data" / "raw" /"customers_data.csv"
inventory_path = project_root/ "data" / "raw" /"inventory_stream"

In [14]:
sales_df = spark.read.csv(str(sales_path), header=True, inferSchema=True)
customers_df = spark.read.csv(str(customers_path), header=True, inferSchema=True)

sales_df.show(5)
customers_df.show(5)


+------+--------+------+--------+-----------+--------+-------+
|txn_id|    date|sku_id|store_id|customer_id|quantity|  price|
+------+--------+------+--------+-----------+--------+-------+
|  T522|20250830|  P001|    X001|       C001|      18|1719.85|
|  T618|20250830|  P001|    X001|       C001|      11|1192.01|
|  T424|20250830|  P001|    X001|       C004|      26|2613.83|
|  T565|20250830|  P001|    X001|       C001|      50|5315.88|
|  T686|20250830|  P001|    X001|       C001|      45|4468.59|
+------+--------+------+--------+-----------+--------+-------+
only showing top 5 rows

+-----------+-------------+------------+-----------+
|customer_id|last_purchase|total_orders|total_spend|
+-----------+-------------+------------+-----------+
|       C001|     20250828|          36|       6516|
|       C002|     20250826|          27|       5292|
|       C003|     20250823|          26|       2574|
|       C004|     20250828|          37|       2331|
+-----------+-------------+----------

We have to define schema for Spark as Spark would not know the schema for inventory as we will be uploading inventory stream batch files incrementally.

In [15]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define Schema
inventory_schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("store_id", StringType(), True),
    StructField("sku_id", StringType(), True),
    StructField("on_stock", IntegerType(), True),
])

# Read streaming JSON with mini-batch drops
inventory_df = spark.readStream \
    .format("json") \
    .schema(inventory_schema) \
    .option("header", "true")  \
    .option("inferSchema", "true") \
    .option("maxFilesPerTrigger",1) \
    .load(str(inventory_path))

inventory_df.printSchema()



root
 |-- timestamp: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- sku_id: string (nullable = true)
 |-- on_stock: integer (nullable = true)



## Processing Layer - Curate the Data
### Daily item-store sales
It tells us how many items were in stock for each SKU at each store per day. This is the foundation for demand calculations.
### Moving Average Demand
It helps us smooth out fluctuations and detect trends. For streaming , we can compute it using window functions.
### Stock-Out Risk Signal
It shows risk signals for Stock-Out.
### RFM Analysis On Customers
It involves Recency which means how recently a customer purchased, Frequency means how often a customer purchased and Monetary means total spend.


# Base function to serve MongoDB

In [16]:
mongo_uri = cfg["mongodb"]["uri"]
mongo_db = cfg["mongodb"]["database"]
collections = cfg["mongodb"]["collections"]

# Threshold stock quantity min
stock_threshold = cfg["thresholds"]["stock_out_qty"]

In [17]:
def write_to_mongo(df, collection_key, mode="overwrite"):
    collection_name = collections[collection_key]
    df.write \
      .format("mongodb") \
      .mode(mode) \
      .option("spark.mongodb.output.uri", mongo_uri) \
      .option("database", mongo_db) \
      .option("collection", collection_name) \
      .save()



# Daily Item-Store Sales And Moving Average OF Quantity

In [18]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

# Aggregate sales at store, date and sku level
sales_df = sales_df.withColumn("date", f.to_date(f.col("date").cast("string"), "yyyyMMdd"))
daily_item_store = sales_df.groupby("date", "store_id", "sku_id") \
    .agg(
    f.sum("quantity").alias("total_qty"),
    f.round(f.sum("price"),2).alias("total_sales"),
    f.round(f.avg("price"), 2).alias("avg_price"),
    f.max("price").alias("max_price"),
    f.min("price").alias("min_price")
)

# Moving Average of Quantity/Demand
window_spec = Window.partitionBy("store_id", "sku_id").orderBy("date").rowsBetween(-6,0)
daily_item_store = daily_item_store.withColumn(
    "moving_avg_qty",
    f.round(f.avg("total_qty").over(window_spec), 2)  # round to 2 decimals
)


# Performance optimizations

# Partition distributes DataFrame across Spark partitions based on store_id and sku_id. This helps reduce shuffling when performing joins or window operation because rows with the same store and SKU end up in the same partition. It improves performance for subsequent operations like window functions or aggregations.
daily_item_store = daily_item_store.repartition("store_id", "sku_id")

# If the following DataFrame is used multiple times, Spark doesn't need to recompute it from scratch each time. This speeds up repeated operations and avoids redundant computations.
daily_item_store = daily_item_store.cache()

daily_item_store.show(5)

write_to_mongo(daily_item_store, "sales") # Pushes curated sales data to MongoDB

+----------+--------+------+---------+-----------+---------+---------+---------+--------------+
|      date|store_id|sku_id|total_qty|total_sales|avg_price|max_price|min_price|moving_avg_qty|
+----------+--------+------+---------+-----------+---------+---------+---------+--------------+
|2025-08-30|    X002|  P002|      161|   32226.96|  4028.37|  8834.62|  1279.78|         161.0|
|2025-08-31|    X002|  P002|      226|   45238.43|   5654.8|  9948.41|  1303.38|         193.5|
|2025-09-01|    X002|  P002|       80|   15406.49|   3081.3|   5835.6|  1475.39|        155.67|
|2025-09-02|    X002|  P002|      214|    41073.2|   5867.6| 10071.92|  2118.02|        170.25|
|2025-09-03|    X002|  P002|      141|   28619.31|  5723.86|  9367.63|  1433.37|         164.4|
+----------+--------+------+---------+-----------+---------+---------+---------+--------------+
only showing top 5 rows



# Stock-Out Risk Signal

In [19]:

inventory_df = inventory_df.withColumn("date", f.to_date("timestamp"))
# Join inventory with daily_item_store to get moving_avg_qty for that SKU and store
inventory_with_sales = inventory_df.join(
    daily_item_store.select("date", "store_id", "sku_id", "moving_avg_qty"),
    on=["store_id", "sku_id", "date"],
    how="left"
)
inventory_with_sales = inventory_with_sales.withColumn(
    "stock_out_signal",
    f.when(f.col("moving_avg_qty").isNotNull() & (f.col("on_stock") <= f.col("moving_avg_qty")), 1)
     .otherwise(0)
)

 # Partition to improve joins and window ops
inventory_with_sales = inventory_with_sales.repartition("store_id", "sku_id")

# Streaming DataFrames cannot be cached.

# Risk signal will be stored in MongoDB. Write method cannot be used in streaming DataFrames. We will have to use writeStream for this DataFrame

# Spark requires checkpoints to keep track of what data has been already processed.

# Path for checkpoint and make sure it exists
checkpoint_path = Path(project_root / "checkpoints" / "inventory_stock")
checkpoint_path.mkdir(parents=True, exist_ok=True)

# Stores risk signal in MongoDB
inventory_with_sales.writeStream \
    .format("mongodb") \
    .option("spark.mongodb.output.uri", mongo_uri) \
    .option("database", mongo_db) \
    .option("collection", "stock") \
    .option("checkpointLocation", checkpoint_path) \
    .outputMode("append") \
    .start()




25/09/20 20:26:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


## Stream mini-stimulation query

In [20]:
console_query = inventory_with_sales.writeStream \
    .format("console") \
    .option("truncate", False) \
    .option("numRows", 20) \
    .option("checkpointLocation", str(checkpoint_path / "console_checkpoint")) \
    .start()

# Stop after 5 batches
batch_count = 0
while batch_count < 5:
    console_query.awaitTermination(1)  # wait 1 sec per loop
    batch_count = len(console_query.recentProgress)

console_query.stop()
print("Stopped stream after 5 batches")


25/09/20 20:26:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/09/20 20:26:35 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:35 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:35 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+------+----------+-------------------+--------+--------------+----------------+
|store_id|sku_id|date      |timestamp          |on_stock|moving_avg_qty|stock_out_signal|
+--------+------+----------+-------------------+--------+--------------+----------------+
|X002    |P002  |2025-08-31|2025-08-31T14:27:13|175     |193.5         |1               |
|X002    |P003  |2025-08-31|2025-08-31T22:54:10|205     |199.5         |0               |
|X001    |P001  |2025-08-31|2025-08-31T16:51:03|252     |258.0         |1               |
|X002    |P001  |2025-08-31|2025-08-31T20:13:28|113     |179.5         |1               |
|X001    |P003  |2025-08-31|2025-08-31T19:38:33|212     |112.0         |0               |
|X001    |P002  |2025-08-31|2025-08-31T07:07:16|146     |240.5         |1               |
+--------+------+----------+-------------------+--------+--------------+----------------+



25/09/20 20:26:36 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:36 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:36 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


-------------------------------------------
Batch: 1
-------------------------------------------
+--------+------+----------+-------------------+--------+--------------+----------------+
|store_id|sku_id|date      |timestamp          |on_stock|moving_avg_qty|stock_out_signal|
+--------+------+----------+-------------------+--------+--------------+----------------+
|X002    |P002  |2025-09-04|2025-09-04T02:08:49|203     |177.17        |0               |
|X002    |P003  |2025-09-04|2025-09-04T03:40:51|114     |183.83        |1               |
|X001    |P001  |2025-09-04|2025-09-04T18:22:08|103     |260.83        |1               |
|X002    |P001  |2025-09-04|2025-09-04T15:02:48|288     |232.67        |0               |
|X001    |P003  |2025-09-04|2025-09-04T05:06:57|216     |182.17        |0               |
|X001    |P002  |2025-09-04|2025-09-04T01:36:11|297     |199.33        |0               |
+--------+------+----------+-------------------+--------+--------------+----------------+



25/09/20 20:26:37 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:37 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:37 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


...

25/09/20 20:26:38 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:38 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:38 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


-------------------------------------------
Batch: 3
-------------------------------------------
+--------+------+----------+-------------------+--------+--------------+----------------+
|store_id|sku_id|date      |timestamp          |on_stock|moving_avg_qty|stock_out_signal|
+--------+------+----------+-------------------+--------+--------------+----------------+
|X002    |P002  |2025-09-01|2025-09-01T13:05:37|235     |155.67        |0               |
|X002    |P003  |2025-09-01|2025-09-01T11:45:53|299     |231.33        |0               |
|X001    |P001  |2025-09-01|2025-09-01T12:00:02|205     |271.33        |1               |
|X002    |P001  |2025-09-01|2025-09-01T20:50:38|241     |213.33        |0               |
|X001    |P003  |2025-09-01|2025-09-01T03:39:49|198     |157.0         |0               |
|X001    |P002  |2025-09-01|2025-09-01T10:38:11|281     |208.0         |0               |
+--------+------+----------+-------------------+--------+--------------+----------------+



25/09/20 20:26:38 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:38 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:38 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.


-------------------------------------------
Batch: 4
-------------------------------------------
+--------+------+----------+-------------------+--------+--------------+----------------+
|store_id|sku_id|date      |timestamp          |on_stock|moving_avg_qty|stock_out_signal|
+--------+------+----------+-------------------+--------+--------------+----------------+
|X002    |P002  |2025-09-02|2025-09-02T05:15:52|107     |170.25        |1               |
|X002    |P003  |2025-09-02|2025-09-02T21:08:38|143     |220.25        |1               |
|X001    |P001  |2025-09-02|2025-09-02T16:03:11|261     |274.5         |1               |
|X002    |P001  |2025-09-02|2025-09-02T19:41:15|237     |232.75        |0               |
|X001    |P003  |2025-09-02|2025-09-02T22:45:06|258     |172.25        |0               |
|X001    |P002  |2025-09-02|2025-09-02T22:11:26|144     |185.0         |1               |
+--------+------+----------+-------------------+--------+--------------+----------------+

St

25/09/20 20:26:39 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
25/09/20 20:26:39 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
